<a href="https://colab.research.google.com/github/Applied-Machine-Learning-2022/project-4-aeds2-uark/blob/ahmed0/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Google LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Video Classification with Pre-Trained Models Project

In this project we will import a pre-existing model that recognizes objects and use the model to identify those objects in a video. We'll edit the video to draw boxes around the identified object, and then we'll reassemble the video so the boxes are shown around objects in the video.

# Exercises

## Exercise 1: Coding

You will process a video frame by frame, identify objects in each frame, and draw a bounding box with a label around each car in the video.
 
Use the [SSD MobileNet V1 Coco](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md) (*ssd_mobilenet_v1_coco*) model. The video you'll process can be found [on Pixabay](https://pixabay.com/videos/cars-motorway-speed-motion-traffic-1900/). The 640x360 version of the video is smallest and easiest to handle, though any size should work since you must scale down the images for processing.
 
Your program should:
 
* Read in a video file (use the one in this colab if you want)
* Load the TensorFlow model linked above
* Loop over each frame of the video
* Scale the frame down to a size the model expects
* Feed the frame to the model
* Loop over detections made by the model
* If the detection score is above some threshold, draw a bounding box onto the frame and put a label in or near the box
* Write the frame back to a new video
 
Some tips:
 
* Processing an entire video is slow, so consider truncating the video or skipping over frames during development. Skipping frames will make the video choppy. But you'll be able to see a wider variety of images than you would with a truncated video with all of the original frames in the clip.
* The model expects a 300x300 image. You'll likely have to scale your frames to fit the model. When you get a bounding box, that box is relative to the scaled image. You'll need to scale the bounding box out to the original image size.
* Don't start by trying to process the video. Instead, capture one frame and work with it until you are happy with your object detection, bounding boxes, and labels. Once you get those done, use the same logic on the other frames of the video.
* The [Coco labels file](https://github.com/nightrome/cocostuff/blob/master/labels.txt) can be used to identify classified objects.
 

### **Student Solution**

In [ ]:
# Read in video file

import cv2 as cv

input_video = cv.VideoCapture('cars.mp4')

h = int(input_video.get(cv.CAP_PROP_FRAME_HEIGHT))
w = int(input_video.get(cv.CAP_PROP_FRAME_WIDTH))
fps = input_video.get(cv.CAP_PROP_FPS)
total_frames = int(input_video.get(cv.CAP_PROP_FRAME_COUNT))

print(f'height: {h}')
print(f'width: {w}')
print(f'frames per second: {fps}')
print(f'total frames: {total_frames}')
print(f'video length (seconds): {total_frames / fps}')

In [ ]:
# Load TF Model

import urllib.request
import os
import tarfile
import shutil

base_url = 'http://download.tensorflow.org/models/object_detection/'
file_name = 'ssd_mobilenet_v1_coco_2018_01_28.tar.gz'

url = base_url + file_name

urllib.request.urlretrieve(url, file_name)

dir_name = file_name[0:-len('.tar.gz')]

if os.path.exists(dir_name):
  shutil.rmtree(dir_name) 

tarfile.open(file_name, 'r:gz').extractall('./')

In [ ]:
# Load the Frozen Graph

import tensorflow as tf

frozen_graph = os.path.join(dir_name, 'frozen_inference_graph.pb')

with tf.io.gfile.GFile(frozen_graph, "rb") as f:
  graph_def = tf.compat.v1.GraphDef()
  loaded = graph_def.ParseFromString(f.read())

outputs = (
  'num_detections:0',
  'detection_classes:0',
  'detection_scores:0',
  'detection_boxes:0',
)

def wrap_graph(graph_def, inputs, outputs, print_graph=False):
  wrapped = tf.compat.v1.wrap_function(
    lambda: tf.compat.v1.import_graph_def(graph_def, name=""), [])

  return wrapped.prune(
    tf.nest.map_structure(wrapped.graph.as_graph_element, inputs),
    tf.nest.map_structure(wrapped.graph.as_graph_element, outputs))
    
model = wrap_graph(graph_def=graph_def,
                   inputs=["image_tensor:0"],
                   outputs=outputs)

In [ ]:
# Create output video with bounding boxes

fourcc = cv.VideoWriter_fourcc(*'mp4v')
output_video = cv.VideoWriter('cars-sampled.mp4', fourcc, fps, (w, h))

# Loop over every 10 frames of the video
for i in range(0, int(input_video.get(cv.CAP_PROP_FRAME_COUNT))):
  input_video.set(cv.CAP_PROP_POS_FRAMES, i)
  ret, frame = input_video.read()
  if not ret:
    raise Exception("Problem reading frame", i, " from video")

  # Feed the frame to the model
  image = cv.resize(frame, (300,300), interpolation = cv.INTER_AREA)
  tensor = tf.convert_to_tensor([image], dtype=tf.uint8)
  detections = model(tensor)

  # Loop over detections and draw bounding boxes
  for i, box in enumerate(detections[3][0]):
    if (detections[1][0][i] == 3 and detections[2][0][i] > .35):
      cv.rectangle(frame, (w*box[1], h*box[0]), (w*box[3], h*box[2]), (255, 0, 0), 2)
      cv.putText(frame, f"Car: {round(float(detections[2][0][i]),2)}", (box[1]*w,  (box[0]*h) - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5,
          [0, 0, 0], 2)

  # Write the frame to new video    
  output_video.write(frame)

input_video.release()
output_video.release()

---

## Exercise 2: Ethical Implications

Even the most basic models have the potential to affect segments of the population in different ways. It is important to consider how your model might positively and negatively affect different types of users.

In this section of the project, you will reflect on the positive and negative implications of your model. Frame the context of your model creation using this narrative:

> The city of Seattle is attempting to reduce traffic congestion in its downtown area. As part of this project, they plan to allow each local driver one free trip to downtown Seattle per week. After that, the driver will have to pay a $50 toll for each extra day per week driven. As an early proof of concept for this project, your team is tasked with using machine learning to correctly identify automobiles on the road. The next phase of the project will involve detecting license plate numbers and then cross-referencing that data with RFID chips that should be mounted in all local drivers' cars.

### **Student Solution**

**Positive Impact**

Your model is trying to solve a problem. Think about who will benefit from that problem being solved and write a brief narrative about how the model will help.

> According to the narrative of our problem, the city of Seattle has a problem with traffic congestion in their downtown area. Since our beginning model will later be developed and implemented into a system that charges frequent drivers, our model can help deter some of the traffic from the downtown area. It will benefit the city financially since it will be able to charge non-local drivers with fines. Our model would also benefit pedestrians in the area so they could move more freely and safely within the downtown area.


**Negative Impact**

Models rarely benefit everyone equally. Think about who might be negatively impacted by the predictions your model is making. This person(s) might not be directly using the model, but they might be impacted indirectly.

> Hypothetically, businesses within the downtown area will be negatively affected. The cost for deliveries would increase since the frequent transportation/logistics companies would have to increase prices to compensate for lost profit due to the fees for going into the downtown area. This wouldn't just hurt businesses in the area, but it would also hurt residents who frequently order food delivery or ride sharing services. The fee for going into the downtown area might serve as a deterance for some delivery drivers since they'll be fined if they frequently go into the downtown area. Our model would charge these services since they make up a part of the frequent traffic within the downtown area.

**Bias**

Models can be biased for many reasons. The bias can come from the data used to build the model (e.g., sampling, data collection methods, available sources) and/or from the interpretation of the predictions generated by the model.

Think of at least two ways bias might have been introduced to your model and explain both below.

> There could be bias in the pre-trained model we used. The pre-trained model could be training data that is missing samples for underrepresented features or categories. For example, the trained data might not have enough examples of different car models, color, number of wheels, size, etc. 

> Another bias could have been added by using data that does not represent more real-world scenarios. In the video we used, the weather was clear and during the day. The reliability of our model could be reduced during harsh weather or during nighttime.

**Changing the Dataset to Mitigate Bias**

Having bias in your dataset is one of the primary ways in which bias is introduced to a machine learning model. Look back at the input data you fed to your model. Think about how you might change something about the data to reduce bias in your model.

What change or changes could you make to reduce the bias in your dataset? Consider the data you have, how and where it was collected, and what other sources of data might be used to reduce bias.

Write a summary of changes that could be made to your input data.

> To reduce bias from input data, we could add data to detect similar vehicles like motorcycles and trucks to make it easier to exclude them from our car predictions. Our model could have been fed different input videos with different weather conditions and cars to yield more realistic results. Studying car features more in-depth to see their effects on detecting a car would help eliminate any features that may possibly induce bias.

**Changing the Model to Mitigate Bias**

Is there any way to reduce bias by changing the model itself? This could include modifying algorithmic choices, tweaking hyperparameters, etc.

Write a brief summary of changes you could make to help reduce bias in your model.

> When the model was trained, it was exclusively trained on things photographed in natural light. As a result, the model may have a more difficult time detecting objects at night. Obtaining the figure's contour may aid in mitigating the bias away from the light bias. Another way to mitigate bias is to base the objects into different class sizes and differentiate from that standpoint.

**Mitigating Bias Downstream**

Models make predictions. Downstream processes make decisions. What processes and/or rules should be in place for people and systems interpreting and acting on the results of your model to reduce bias? Describe these rules and/or processes below.

> We know as a collective that the model can only detect things that have been trained, and people and systems interested in the model should be aware that any other items that have not been trained are foreign to our model. In our situation, the model was trained with things that are prevalent in the United States; but, if we were to cast our model in, say, Japan or another country, they may have a lot of foreign items that are not in our model.  Another issue is the weather conditions , gathering more information images regarding various weather circumstances might significantly enhance our model in that regard.

---